## Data Preprocessing

In [1]:
# installs
!pip install torch==2.0.0
!pip install tensorflow
# !pip install concrete-ml transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.9 MB/s eta 0:00:00


In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
# # Load the dataset
bully_index = [
    "not_cyberbullying",
    "cyberbullying"
]
bully_columns = [
    "tweet",
    "class",
]
# https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification
bully_df = pd.read_csv('/content/drive/MyDrive/PrivacySystems/ss_bully.csv')
print(bully_df.value_counts("tagging"))
# def class_shrink(x):
#   if x != "not_cyberbullying": 
#     return 1
#   return 0
# bully_df["class"] = bully_df["class"].apply(class_shrink)
bully_df

tagging
0    12179
1     7822
dtype: int64


,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1
...,...,...
19996,I dont. But what is complaining about it goi...,0
19997,Bahah yeah i&;m totally just gonna&; get pis...,0
19998,hahahahaha >:) im evil mwahahahahahahahahaha,0
19999,What&;s something unique about Ohio? :),0


In [79]:
bully_df.value_counts("tagging")

tagging
0    12179
1     7822
dtype: int64

<b>Balancing Data: </b><br/><br/>
cyberbullying --------- 39748 <br/>
not_cyberbullying ---- 7945 <br/><br/>
Our dataset is imbalanced favoring datapoints that are classifyable as cyber bullying. It will skew the models predictions towrds classifying entries as "cyberbullying"(depending on the model). However, I am fine with this as the cost of false negatives is higher.


In [80]:
# import transformers
import torch
from transformers import TFAutoModel,AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
input = tokenizer("Luke, I am your father",padding = True, truncation =True, return_tensors="tf")
input

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=
array([[    0, 34899,     6,    38,   524,   110,  1150,     2]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

## Train Model 

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
# Train test data split for our model
X_train, X_test, y_train, y_test = train_test_split(bully_df.drop(columns=["tagging"]),bully_df["tagging"],
    test_size=0.2, random_state=42,stratify = bully_df["tagging"])
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train["comments"])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = LinearSVC().fit(X_train_tfidf, y_train)

In [115]:
def model_pedict(text):
  return bully_index[clf.predict(count_vect.transform([text]))[0]]

In [116]:
# example prediction
model_pedict("How's mom?")

'not_cyberbullying'

In [91]:
clf.score(count_vect.transform(X_test["comments"]),y_test)

0.6580854786303424

In [97]:
# Pickle Model
pickle.dump(clf, open('model.pkl','wb'))
pickle.dump(count_vect,open('count_vect.pkl','wb'))

## Analyze Model 

Citations:
<br/>
https://huggingface.co/blog/sentiment-analysis-fhe
<br/>
https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f